In [1]:
#!pip install openai

In [3]:
import openai

# Single Agent with GPT 3.5 maintains State and uses it to generate prompts
class ConverestionAgent:
  def __init__(self, api_key, role):
    self.api_key = api_key
    self.role = role
    self.conversation_histroy = []
    openai.api_key = self.api_key

  def add_to_conversation(self, message):
    self.conversation_history.append(message)

  def generate_prompt(self):
    prompt    = f'The following is the conversation with an AI playing the role of {self.role}.\n'
    for message in self.conversation_history:
      prompt += message + "\n"
      return prompt

  def perform_task(self, user_message):
    self.add_to_conversation(f'User: {user_message}')
    prompt = self.generate_prompt()
    response = openai.Completion.create(engine = 'text-davinci-003',
                                        prompt = prompt,
                                        max_tokens = 500)
    self.add_to_conversation(f'AI: {response.choices[0].text.strip()}')
    return response.choices[0].text.strip()


An actual Agent could be much more comple, with additional logic for interpretting the LLM's response, managing multiple tasks, interacting with other agents, and more

# 2 Agents


In [ ]:
import openai

class UserAgent:
  def __init__(self, api_key):
    self.api_key = api_key
    openai.api_key = self.api_key

  def ask_question(self, question):
    return f'User: {question}'

class HelpdeskAgent:
  def __init__(self, api_key):
    self.api_key = api_key
    self.conversation_history = []
    openai.api_key = self.api_key

  def add_to_conervsation(self,message):
    self.conversation_history.append(message)


  def generate_prompt(self):
    prompt    = f'The following is the conversation with an HelpDesk AI '
    for message in self.conversation_history:
      prompt += message + "\n"
      return prompt

  def escalate_issue(self, issue):
    print(f'Issue escalted: {issue}')


  def answer_question(self, user_message):
    self.add_to_conversation(user_message)
    prompt = self.generate_prompt()
    response = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens=100)
    ai_response = response.choices[0].text.strip()
    self.add_to_conversation(f"AI: {ai_response} ")

    if 'I"m not sure' in ai_response:
      self.escalate_issue(user_message)

    return ai_response



user_agent = UserAgent("openai-api-key")
helpdesk_agent = HelpdeskAgent("openai-api-key ")

question = user_agent.ask_question("What is API?")
answer = helpdesk_agent.answer_question(question)



# 3 Agent : Discussion


In [5]:
import openai

class NegotiationAgent:

  def __init__(self, api_key, name):
    self.api_key = api_key
    self.name = name
    self.conversation_history = []
    openai.api_key = self.api_key

  def add_to_conervsation(self, message):
    self.conversation_history.append(message)

  def generate_prompt(self):
    prompt    = f'The following is the conversation with an HelpDesk AI '
    for message in self.conversation_history:
      prompt += message + "\n"
      return prompt

  def negotiate(self, message):
    self.add_to_conversation(message)
    prompt = self.generate_prompt()
    response = openai.Completion.create(engine = 'text-davinci-003', prompt = prompt, max_tokens = 100)
    ai_response = response.choices[0].text.strip()
    self.add_to_conversation(ai_response)
    return ai_response

class MediatorAgent:
  def __intit__(self, api_key):
    self.api_key = api_key
    self.conversation_hostory = []
    openai.api_key = self.api_key

  def add_to_conervsation(self, message):
    self.conversation_history.append(message)

  def generate_prompt(self):
    prompt    = f'The following is the conversation with an HelpDesk AI '
    for message in self.conversation_history:
      prompt += message + "\n"
      return prompt

  def mediate(self, negotiation_agents):
    for agent in negotiation_agents:
      self.add_to_conversation(agent.conversation_history[-1])
    prompt = self.generate_prompt()
    response = response.choices[0].text.strip()
    ai_response = response.choices[0].text.strip()
    self.add_to_conversation(ai_response)
    return ai_response

agent1 = NegotiationAgent("api-key", 'Party1')
agent2 = NegotiationAgent("api-key", 'Party2')
mediator_agent = MediatorAgent("api-key")

negotiation1 = agent1.negotiate("I want to buy 100 Iphone in 1000 rs each")
negotiation2 = agent2.negotiate("We want to sell a Iphone rs 3000 ")

mediation = mediator_agent([agent1,agent2])



Agents that create temporally new Agents

In [ ]:
# @title
import openai

class TaskAgent:
  def __init__(self, api_key, task):
    self.api_key = api_key
    self.task = task
    openai.api_key = self.api_key


  def perform_task(self):
       prompt = f'Ai, please perform the following task : {self.task}'
       response = openai.Completion.create(engine='text-davinci-003', prompt = prompt, max_tokens=100)
       return response.choices[0].text.strip()


class ControlAgent:
  def __init__(self, api_key):
    self.api_key = api_key

  def create_agent(self,task):
    return TaskAgent(self.api_key,task)

  def manage_task(self, task):
    agent = self.create_agent(task)
    result = agent.perform_task()
    return result

control_agent = ControlAgent('openai-api-key')

result = control_agent.manage_task("Translate the following text to French: 'Hello world'")


Why to create the Agents?
# Multi Agents LLMs


In [ ]:
import openai

openai.api_key = 'api_key'

class Agent:
  def __init__(self, role, can_create_agents, can_halt_agents, plugins):
    self.role = role
    self.can_create_agents = can_create_agents
    self.can_halt_agents = can_halt_agents
    self.plugins = plugins
    self.state = None

  def generate_response(self, prompt):
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-0613',
        messages = [
            {"role" : "system", "content" : f"You are a {self.role}"},
            {"role" : "user", "content" : prompt}
        ],
        functions = [
            {
                "name" : self.role,
                "description": f"Generates a response based on the role of {self.role}",
                "input_schema": {
                    "type" : "object",
                    "properties": {
                        "prompt": {"type": "string", "description": "The prompt for the Agent"},
                    },
                    "reuired": ["prompt"],
                },
                "output_schema": {
                    "type" : "object",
                    "properties": {
                        "response": {"type": "string", "description": "The generated response"},
                    },
                    "reuired": ["response"],
                },
            }
        ]
    )
    assistant_messages = [message['content'] for message in response['messages'] if message['role'] == 'assistant']

    return assistant_messages[-1] if assistant_messages else None

    def create_agent(self, role, can_create_agents, can_halt_agents, plugins):
      if self.can_create_agents:
        return Agent(role, can_create_agents, can_halt_agents, plugins)
      else:
        raise Exception("This agent does not have the ability to create other agents")

     def halt_agent(self, agent):
      if self.can_create_agents:
        return Agent(role, can_create_agents, can_halt_agents, plugins)
      else:
        raise Exception("This agent does not have the ability to create other agents")


emotional_agent = Agent("Emotional Agent", True, True, ["Emotional language Generation", "Sentiment Analysis"])
factual_agent = Agent("Factual Agent", True, True, ["Fact-Based language Generation", "Logiacl reasoning"])
moderator_agent = Agent("Moderator Agent", True, True, ["Time Management", "Topic Introduction"])
rhetoric_agent = emotional_agent.create_agent(" Agent", False, False, ["Emotional language Generation", "Sentiment Analysis"])

